In [1]:
import pandas as pd
import numpy as np
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
lemaoi = WordNetLemmatizer()
import pickle

In [35]:
def word_sequence(n_most_words, x, max_len):
    tokeniz = Tokenizer(num_words=n_most_words)
    tokeniz.fit_on_texts(x.values)
    sequen = tokeniz.texts_to_sequences(x.values)
    word_index = tokeniz.word_index
    X = pad_sequences(sequen, maxlen=max_len)
    return X, word_index

In [3]:
def under_sample(raw):
    posi = raw_data.loc[raw_data['sentiment']=='Positive'][:100]
    Neu = raw_data.loc[raw_data['sentiment']=='Neutral'][:100]
    Nega = raw_data.loc[raw_data['sentiment']=='Negative']
    traning_data = pd.concat([posi, Neu, Nega])
    traning_data.reset_index(inplace=True)
    traning_data['sentiment'].replace({'Positive':1,'Negative':-1,'Neutral':0}, inplace=True)
    return traning_data

In [3]:
def over_sample(raw):
    posi = raw_data.loc[raw_data['sentiment']=='Positive'][:1000]
    Neu = raw_data.loc[raw_data['sentiment']=='Neutral']
    Nega = raw_data.loc[raw_data['sentiment']=='Negative']
    Neu_new = pd.concat([Neu,Neu,Neu,Neu,Neu,Neu], axis=0)
    Nega_new = pd.concat([Nega,Nega,Nega,Nega,Nega,Nega,Nega,Nega,Nega,Nega])
    traning_data = pd.concat([posi, Neu_new, Nega_new])
    traning_data.reset_index(inplace=True)
    traning_data['sentiment'].replace({'Positive':1,'Negative':-1,'Neutral':0}, inplace=True)
    return traning_data
    

In [4]:
def pre_processing(x):
    x = x.lower()
    x = re.sub('[^a-z0-9 ]','',x)
    result = []
    for i in x.split():
        if i in stopwords.words('english'):
            pass
        else:
            if len(i) <3:
                pass
            else:
                result.append(lemaoi.lemmatize(i))
    res = ' '.join(result)
    res = re.sub('\d{2,}\w+','',res)
    return res

In [5]:
raw_data = pd.read_csv('train_data.csv')
raw_data.head(5)

,name,brand,categories,primaryCategories,reviews.date,reviews.text,reviews.title,sentiment
0,"All-New Fire HD 8 Tablet, 8"" HD Display, Wi-Fi...",Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Electronics,2016-12-26T00:00:00.000Z,Purchased on Black FridayPros - Great Price (e...,Powerful tablet,Positive
1,Amazon - Echo Plus w/ Built-In Hub - Silver,Amazon,"Amazon Echo,Smart Home,Networking,Home & Tools...","Electronics,Hardware",2018-01-17T00:00:00.000Z,I purchased two Amazon in Echo Plus and two do...,Amazon Echo Plus AWESOME,Positive
2,Amazon Echo Show Alexa-enabled Bluetooth Speak...,Amazon,"Amazon Echo,Virtual Assistant Speakers,Electro...","Electronics,Hardware",2017-12-20T00:00:00.000Z,Just an average Alexa option. Does show a few ...,Average,Neutral
3,"Fire HD 10 Tablet, 10.1 HD Display, Wi-Fi, 16 ...",Amazon,"eBook Readers,Fire Tablets,Electronics Feature...","Office Supplies,Electronics",2017-08-04T00:00:00.000Z,"very good product. Exactly what I wanted, and ...",Greattttttt,Positive
4,"Brand New Amazon Kindle Fire 16gb 7"" Ips Displ...",Amazon,"Computers/Tablets & Networking,Tablets & eBook...",Electronics,2017-01-23T00:00:00.000Z,This is the 3rd one I've purchased. I've bough...,Very durable!,Positive


In [6]:
train_frame = over_sample(raw_data)
train_frame.columns

Index(['index', 'name', 'brand', 'categories', 'primaryCategories',
       'reviews.date', 'reviews.text', 'reviews.title', 'sentiment'],
      dtype='object')

In [7]:
train_frame['data'] = train_frame['reviews.text'] + ' '+ train_frame['reviews.title']
data_to_train = train_frame.loc[:,['data','sentiment']]
data_to_train.head(5)

,data,sentiment
0,Purchased on Black FridayPros - Great Price (e...,1
1,I purchased two Amazon in Echo Plus and two do...,1
2,"very good product. Exactly what I wanted, and ...",1
3,This is the 3rd one I've purchased. I've bough...,1
4,This is a great product. Light weight. I wish ...,1


In [8]:
data_to_train['sentiment'].value_counts()

 1    1000
 0     948
-1     930
Name: sentiment, dtype: int64

In [9]:
data_to_train.dropna(axis=0, inplace=True)

In [10]:
data_to_train['cleaned_data'] = data_to_train['data'].apply(lambda x: pre_processing(x))
data_to_train

,data,sentiment,cleaned_data
0,Purchased on Black FridayPros - Great Price (e...,1,purchased black fridaypros great price even sa...
1,I purchased two Amazon in Echo Plus and two do...,1,purchased two amazon echo plus two dot plus fo...
2,"very good product. Exactly what I wanted, and ...",1,good product exactly wanted good price greattt...
3,This is the 3rd one I've purchased. I've bough...,1,3rd one ive purchased ive bought one niece cas...
4,This is a great product. Light weight. I wish ...,1,great product light weight wish wifi download ...
...,...,...,...
2873,"Too proprietary, the apps my daughter liked I ...",-1,proprietary apps daughter liked could install ...
2874,This tablet could be so much better if the app...,-1,tablet could much better apps actually worked ...
2875,"Waste of money, ask Alexa a question and you a...",-1,waste money ask alexa question lucky get answe...
2876,"The tap is a great concept, I love my echo so ...",-1,tap great concept love echo portable one much ...


In [11]:
Y = to_categorical(data_to_train['sentiment'], num_classes=3)
Y

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [16]:
vocab = 2900
max_len_for_sen = 30
Embedding = 50

In [36]:
X, word_i = word_sequence(vocab,data_to_train['cleaned_data'],max_len_for_sen)

In [18]:
max(word_i.values())

2921

In [19]:
X

array([[1434,  351, 1845, ...,   94,  627,    1],
       [1850,   53,  492, ...,   12,  116,  157],
       [   0,    0,    0, ...,    3,   24, 1853],
       ...,
       [   0,    0,    0, ...,  117,  105,   94],
       [   0,    0,    0, ...,    3,  412,   43],
       [   0,    0,    0, ...,   73,   56, 1091]])

In [20]:
X[0]

array([1434,  351, 1845, 1846,    4,  260,    4,   29, 1847,    1,  530,
        306,  168,  223,   14,  419,    6,    4,  156,   81, 1848,  306,
         71,   34, 1849,    2,    1,   94,  627,    1])

In [21]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.3, random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(2013, 30)
(864, 30)
(2013, 3)
(864, 3)


In [22]:
x_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0, 1257, 1258, 1259, 1160,  186,  231,  675, 1209,
         18,   68,  190,  251,   87,  708,  173, 1260])

In [23]:
amazon = models.Sequential()
amazon.add(layers.Embedding(vocab, 40, input_length=max_len_for_sen))
amazon.add(layers.SpatialDropout1D(0.2))
amazon.add(layers.LSTM(100))
amazon.add(layers.Dense(3, activation='sigmoid'))
print(amazon.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 40)            116000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 30, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 3)                 303       
Total params: 172,703
Trainable params: 172,703
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
amazon.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [25]:
amazon.fit(x_train,y_train,batch_size=50,epochs=20)

Train on 2013 samples
Epoch 1/20
2013/2013 [==============================] - 7s 3ms/sample - loss: 1.0463 - acc: 0.4486
Epoch 2/20
2013/2013 [==============================] - 1s 745us/sample - loss: 0.6636 - acc: 0.7273
Epoch 3/20
2013/2013 [==============================] - 2s 771us/sample - loss: 0.3747 - acc: 0.8594
Epoch 4/20
2013/2013 [==============================] - 2s 752us/sample - loss: 0.2711 - acc: 0.9126
Epoch 5/20
2013/2013 [==============================] - 2s 768us/sample - loss: 0.1568 - acc: 0.9568
Epoch 6/20
2013/2013 [==============================] - 2s 762us/sample - loss: 0.0957 - acc: 0.9752
Epoch 7/20
2013/2013 [==============================] - 2s 803us/sample - loss: 0.0632 - acc: 0.9806
Epoch 8/20
2013/2013 [==============================] - 2s 818us/sample - loss: 0.0416 - acc: 0.9906
Epoch 9/20
2013/2013 [==============================] - 2s 810us/sample - loss: 0.0243 - acc: 0.9940
Epoch 10/20
2013/2013 [==============================] - 2s 763us/sampl

In [26]:
amazon.evaluate(x_test,y_test)

864/864 [==============================] - 1s 769us/sample - loss: 0.1700 - acc: 0.9572


[0.16999183223513817, 0.9571759]

In [37]:
amazon.save('amazon.h5')

In [34]:
pickle.dump(token_,open('tokenizer_amazon.pickle','wb'))

In [43]:
test_cr = pd.read_csv('test_data.csv')

In [44]:
test_cr.dropna(axis=0, inplace=True)

In [46]:
test_cr['data'] = test_cr['reviews.text'] + ' '+ test_cr['reviews.title']

In [47]:
test_cr['cleaned_data'] = test_cr['data'].apply(lambda x: pre_processing(x))
test_cr

,name,brand,categories,primaryCategories,reviews.date,reviews.text,reviews.title,data,cleaned_data
0,"Fire Tablet, 7 Display, Wi-Fi, 16 GB - Include...",Amazon,"Fire Tablets,Computers/Tablets & Networking,Ta...",Electronics,2016-05-23T00:00:00.000Z,Amazon kindle fire has a lot of free app and c...,very handy device,Amazon kindle fire has a lot of free app and c...,amazon kindle fire lot free app used one want ...
1,Amazon Echo Show Alexa-enabled Bluetooth Speak...,Amazon,"Computers,Amazon Echo,Virtual Assistant Speake...","Electronics,Hardware",2018-01-02T00:00:00.000Z,The Echo Show is a great addition to the Amazo...,Another winner from Amazon,The Echo Show is a great addition to the Amazo...,echo show great addition amazon family work li...
2,"All-New Fire HD 8 Tablet, 8"" HD Display, Wi-Fi...",Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Electronics,2017-01-02T00:00:00.000Z,Great value from Best Buy. Bought at Christmas...,simple to use and reliable so far,Great value from Best Buy. Bought at Christmas...,great value best buy bought christmas sale sim...
3,"Brand New Amazon Kindle Fire 16gb 7"" Ips Displ...",Amazon,"Computers/Tablets & Networking,Tablets & eBook...",Electronics,2017-03-25T00:00:00.000Z,"I use mine for email, Facebook ,games and to g...",Love it!!!,"I use mine for email, Facebook ,games and to g...",use mine email facebook game line also loaded ...
4,Amazon Echo Show Alexa-enabled Bluetooth Speak...,Amazon,"Computers,Amazon Echo,Virtual Assistant Speake...","Electronics,Hardware",2017-11-15T00:00:00.000Z,This is a fantastic item & the person I bought...,Fantastic!,This is a fantastic item & the person I bought...,fantastic item person bought love fantastic
...,...,...,...,...,...,...,...,...,...
995,Amazon Echo Show Alexa-enabled Bluetooth Speak...,Amazon,"Computers,Amazon Echo,Virtual Assistant Speake...","Electronics,Hardware",2017-12-07T18:06:07.000Z,We love Alexa! Love being able to watch news b...,Awesome,We love Alexa! Love being able to watch news b...,love alexa love able watch news briefing seein...
996,Amazon Tap - Alexa-Enabled Portable Bluetooth ...,Amazon,"Amazon Echo,Home Theater & Audio,MP3 MP4 Playe...",Electronics,2017-01-23T00:00:00.000Z,Speaker is pretty loud and I love that I can t...,Good Bluetooth device,Speaker is pretty loud and I love that I can t...,speaker pretty loud love take downside hold ho...
997,"Fire HD 8 Tablet with Alexa, 8"" HD Display, 32...",Amazon,"Tablets,Fire Tablets,Computers & Tablets,All T...",Electronics,2017-01-18T00:00:00.000Z,Bought 2 of these for my 6 and 8 yr old and it...,good product for kids,Bought 2 of these for my 6 and 8 yr old and it...,bought old said came storage actual storage l...
998,"All-New Fire HD 8 Tablet, 8"" HD Display, Wi-Fi...",Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Electronics,2016-12-12T00:00:00.000Z,Was told by sales person I could come back in ...,Was misled on price match. Item is okay.,Was told by sales person I could come back in ...,told sale person could come back week receipt ...


In [48]:
y_test_cr,wi= word_sequence(vocab,test_cr['cleaned_data'],max_len_for_sen)

In [51]:
y_pred = amazon.predict_classes(y_test_cr)

In [52]:
y_pred

array([1, 0, 1, 1, 1, 1, 2, 0, 0, 1, 1, 1, 2, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 2, 1, 0, 0, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1,
       0, 0, 2, 1, 0, 2, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       2, 1, 2, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 2, 1, 0, 0, 1,
       1, 0, 0, 1, 2, 1, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 2, 0, 1, 1, 0, 0, 1, 1, 0, 1, 2, 1, 1, 0, 1, 1, 1, 2, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1,
       1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 0,
       2, 0, 2, 1, 2, 1, 1, 0, 0, 1, 1, 1, 1, 1, 2, 1, 0, 2, 1, 1, 2, 1,
       0, 1, 1, 1, 2, 2, 0, 0, 1, 0, 1, 0, 2, 2, 1, 1, 1, 2, 0, 1, 0, 1,
       2, 2, 1, 0, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1,